# Machine Learning and Artificial Intelligence
## Stefano Brilli s249914 - homework 1

### 1 Analysis definition
In this homework, I will use the tecnique called Principal Components Analysis (PCA) to reduce the dimensionality of the dataset provided. In particular, we want to study how PCA works on images compression.

#### 1.1 PCA briefly explanation
Principal Components Analysis (PCA) is a tecnique that allows us to summarize a large data set with a smaller number of representative variables that collectively explain most of the variability of out original set.
It is useful when we have a large data set, in which each observation has many features (or properties). In this scenario, it is difficult to represent data onto just 2 dimensional space and so it’s difficult to observe how data are correlated among them.
The tecnique uses the eigenvectors and eigenvalues of the correlation matrix related to our set to reproject the observations in a new lower dimensional space. The aim of this method is to reduce the dimensionality by keeping as more information as possible.

#### 1.2 Tools used
The programming language used is Python (version 3.7).
The main libraries used to perform these analysis are NumPy for using arrays (http://www.numpy.org/), Sklearn for using machine learning tecniques (http://scikit-learn.org/stable/documentation.html), MatPlotLib for plotting images (https://matplotlib.org/) and Pillow to handle images (PIL, https://pillow.readthedocs.io/en/3.1.x/index.html). So, the first thing to do is to import the above libraries in order to use them.

In [ ]:
from PIL import Image
import numpy as np
import os
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import matplotlib.colors as colo
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

#### 1.3 Data preparation
The dataset contains 1087 samples with 3x227x227 sample size. The goal is to build a matrix NxM, where each row represents an image and each columns represent a feature. Since each image has three dimensions, I have to flat them to obtain a vector representing the pixels and it can be done by using the method ravel() of numpy library. The following function loads in memory the images.

In [ ]:
# This method opens each class folder and gets raw pixels of each image
def getData(directory_name, x, label, y):
    directory = os.fsencode(directory_name)
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        i = Image.open(directory_name + filename)
        x.extend(np.asarray(i))
        global count
        y.insert(count, label)
        count += 1
        global numbers
        numbers[label]  = numbers[label] + 1

In the function getData(...) I have filled some vectors such as x, y and numbers.
The latter one is used to count how many images there are for each class label. As we will see soon, the dataset isn't divided perfectly in four subsets according to the class label. This fact will bring some consequences to the analysis.

In [ ]:
# vector representing how many elements there are in each folder
# 0:dog 1:guitar 2:house 3:person
numbers = [0, 0, 0, 0]
x = [] # list of items
y = [] # list of labels
count = 0
X_t = [] # here I save the eigenvectors of my dataset according to the number of PC

At this point it's possible to load the images in memory, before starting with the next part. To do that, I have declared the path of the dataset and called four times the function stated before.
After loading images, I've used the reshape() method to flat each image as a matrix's row.

In [ ]:
# Dataset with subfolders
rootFolder = '/home/stefano/Documenti/Politecnico/Magistrale/2 Anno/ML/Homework/#1/PACS_homework/' # root images folder
folder1 = 'dog/'
folder2 = 'guitar/'
folder3 = 'house/'
folder4 = 'person/'

getData(rootFolder + folder1, x, 0, y) # subset of dog images
getData(rootFolder + folder2, x, 1, y) # subset of guitar images
getData(rootFolder + folder3, x, 2, y) # subset of house images
getData(rootFolder + folder4, x, 3, y) # subset of person images

x = np.asarray(x, dtype=np.float64) # all 3D images
x_r = np.reshape(x, (1087,154587)) # vectorial representation of matrix

### 2 Principal Components Visualization

Before proceding, I've to perform a stardardization on data. This is useful in particular for classification, but also for plotting data in a simple reference system. The standardization consists of subtracting the dataset's mean from any sample, and dividing each one by the standard deviation. Any feature of the resulting dataset will be then in the form 0 <= xi <= 1.

In [ ]:
# To turn back to original values distribution I've to keep the scaler object in a variable
scaler = StandardScaler()
x_r = scaler.fit_transform(x_r)